In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle competitions download -c ashrae-energy-prediction

 75% 66.0M/88.4M [00:00<00:00, 149MB/s]
100% 88.4M/88.4M [00:00<00:00, 180MB/s]
 95% 159M/167M [00:00<00:00, 201MB/s]
100% 167M/167M [00:00<00:00, 204MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 179MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 45.9MB/s]
 94% 112M/120M [00:00<00:00, 183MB/s] 
100% 120M/120M [00:00<00:00, 202MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 148MB/s]


In [ ]:
pip install pyunpack

In [ ]:
pip install patool

     |████████████████████████████████| 81kB 10.3MB/s 


In [ ]:
from pyunpack import Archive
Archive('/content/test.csv.zip').extractall('/content/')
Archive('/content/weather_test.csv.zip').extractall('/content/')

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time

# Read data...
root = '/content'

test_df = (pd.read_csv(os.path.join(root, 'test.csv')))
weather_test_df = (pd.read_csv(os.path.join(root, 'weather_test.csv')))
building_meta_df = (pd.read_csv(os.path.join(root, 'building_metadata.csv')))

CPU times: user 10.1 s, sys: 1.66 s, total: 11.7 s
Wall time: 11.7 s


In [ ]:
#https://www.kaggle.com/corochann/ashrae-feather-format-for-fast-loading
#feather is used to reduce the size and load the dataset quickly
%%time

test_df.to_feather('test.feather')
weather_test_df.to_feather('weather_test.feather')
building_meta_df.to_feather('building_metadata.feather')

CPU times: user 2.46 s, sys: 1.39 s, total: 3.85 s
Wall time: 4.04 s


In [ ]:
%%time

test_df = pd.read_feather('test.feather')
weather_test_df = pd.read_feather('weather_test.feather')
building_meta_df = pd.read_feather('building_metadata.feather')

CPU times: user 2.15 s, sys: 1.4 s, total: 3.55 s
Wall time: 2.79 s


> Function to reduce memory usage

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
test_df = reduce_mem_usage(test_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [ ]:
test_data = test_df.merge(building_meta_df,how='left',on='building_id')
test_data = test_data.merge(weather_test_df,how='left',on=['site_id','timestamp']) 

In [ ]:
# converting timestamp to datetime object
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])

#Dealing with Missing data


In [ ]:
features_with_na = [feature for feature in test_data.columns if test_data[feature].isnull().sum()>1]
for feature in features_with_na:
    print(feature+' : '+str(np.round(test_data[feature].isnull().mean(),4)))

year_built : 0.5899
floor_count : 0.8261
air_temperature : 0.0053
cloud_coverage : 0.4687
dew_temperature : 0.0063
precip_depth_1_hr : 0.1871
sea_level_pressure : 0.0604
wind_direction : 0.0714
wind_speed : 0.0072


* Remove features which are having >80 of missing data. So, we remove floor_count feature.<br>
* For remaining missing values features we have to impute.

In [ ]:
#removing floor_count feature from data
test_data.drop('floor_count',axis=1,inplace=True)
test_data.reset_index(inplace=True)

In [ ]:
test_data['day'] = pd.to_datetime(test_data['timestamp']).dt.dayofyear
test_data['month'] = pd.to_datetime(test_data['timestamp']).dt.month

In [ ]:
test_data[['dew_temperature','cloud_coverage','year_built','air_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']] = test_data[['dew_temperature','cloud_coverage','year_built','air_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']].astype(np.float32)

In [ ]:
test_data = reduce_mem_usage(test_data)

Mem. usage decreased to 2028.06 Mb (39.3% reduction)


In [ ]:
#filling nan values with interpolation method
test_data['air_temperature'] = test_data['air_temperature'].interpolate(method='linear').ffill().bfill()
test_data['dew_temperature'] = test_data['dew_temperature'].interpolate(method='linear').ffill().bfill()
test_data['cloud_coverage'] = test_data['cloud_coverage'].interpolate(method='linear').ffill().bfill()
test_data['wind_speed'] = test_data['wind_speed'].interpolate(method='linear').ffill().bfill()
test_data['wind_direction'] = test_data['wind_direction'].interpolate(method='linear').ffill().bfill()
test_data['sea_level_pressure'] = test_data['sea_level_pressure'].interpolate(method='linear').ffill().bfill()
test_data['precip_depth_1_hr'] = test_data['precip_depth_1_hr'].interpolate(method='linear').ffill().bfill()

In [ ]:
#for imputing year interpolation is not a good idea. so, by taking median grouping it by with site_id,month,day.
fill=test_data.groupby(['site_id','day','month'])['year_built'].median().reset_index()
fill.rename(columns={'year_built':'filler'},inplace=True)
fill['filler'].fillna(method='ffill',inplace=True)
test_data=test_data.merge(fill,how='left',on=['site_id','day','month'])
test_data['year_built'].fillna(test_data['filler'],inplace=True)
test_data.drop(labels=['filler'],axis=1,inplace=True)

##LabelEncoding primary_use feature

In [ ]:
le = LabelEncoder()
test_data['primary_use'] = le.fit_transform(test_data['primary_use'])

## Alignment of air_temperature according to local_temperature

In [ ]:
#air_temperature maximizes around 19:00PM to 23:00PM for 13 different sites which needs to be corrected 
#so that the model can best identify the energy pattern according to the temperature variations over the hour

align_hours = {0:5,2:9,3:5,4:8,6:6,7:6,8:5,9:7,10:8,11:6,13:7,14:5,15:5} 
def alignment(data,align_hours):
    data.reset_index(inplace=True)
    data['timestamp_alignment'] = data['timestamp'] - timedelta(hours=align_hours)
    data_air_temp = data[['timestamp_alignment','building_id','meter','air_temperature']].copy()
    data.drop(['air_temperature','timestamp_alignment'],axis=1,inplace=True)
    data['air_temperature_alignment'] = data_air_temp[data_air_temp['timestamp_alignment'].isin(data['timestamp'])].reset_index(drop=True)['air_temperature']
    data['air_temperature_alignment'] = data['air_temperature_alignment'].interpolate()
    data.rename(columns={'air_temperature_alignment':'air_temperature','timestamp_alignment':'timestamp'},inplace=True)
    data.drop(['level_0','index'],axis=1,inplace=True)
    return data

In [ ]:
def no_alignment(data):
    data.reset_index(inplace=True)
    data.drop(['level_0','index'],axis=1,inplace=True)
    return data

In [ ]:
air_temp_site_0_alignment = alignment(test_data.query(f'site_id==0'),5)
air_temp_site_1_alignment = no_alignment(test_data.query(f'site_id==1'))
air_temp_site_2_alignment = alignment(test_data.query(f'site_id==2'),9)
air_temp_site_3_alignment = alignment(test_data.query(f'site_id==3'),5)
air_temp_site_4_alignment = alignment(test_data.query(f'site_id==4'),8)
air_temp_site_5_alignment = no_alignment(test_data.query(f'site_id==5'))
air_temp_site_6_alignment = alignment(test_data.query(f'site_id==6'),6)
air_temp_site_7_alignment = alignment(test_data.query(f'site_id==7'),6)
air_temp_site_8_alignment = alignment(test_data.query(f'site_id==8'),5)
air_temp_site_9_alignment = alignment(test_data.query(f'site_id==9'),7)
air_temp_site_10_alignment = alignment(test_data.query(f'site_id==10'),8)
air_temp_site_11_alignment = alignment(test_data.query(f'site_id==11'),6)
air_temp_site_12_alignment = no_alignment(test_data.query(f'site_id==12'))
air_temp_site_13_alignment = alignment(test_data.query(f'site_id==13'),7)
air_temp_site_14_alignment = alignment(test_data.query(f'site_id==14'),5)
air_temp_site_15_alignment = alignment(test_data.query(f'site_id==15'),5)

In [ ]:
test_data_final = pd.concat([air_temp_site_0_alignment,air_temp_site_1_alignment,air_temp_site_2_alignment,air_temp_site_3_alignment,air_temp_site_4_alignment,air_temp_site_5_alignment,air_temp_site_6_alignment,air_temp_site_7_alignment,air_temp_site_8_alignment,air_temp_site_9_alignment,air_temp_site_10_alignment,air_temp_site_11_alignment,air_temp_site_12_alignment,air_temp_site_13_alignment,air_temp_site_14_alignment,air_temp_site_15_alignment],axis=0)

In [ ]:
#sort datapoints based on timestamp,building_id
test_data_final = test_data_final.sort_values(['timestamp','building_id'])

In [ ]:
test_data_final['hour'] = test_data_final['timestamp'].dt.hour
test_data_final['weekday'] = test_data_final['timestamp'].dt.weekday

* During day time in working hours the meter readings are high.
* when compared the days of week the weekdays have more meter readings than weekands. 

In [ ]:
test_data_final['is_winter_month']=(test_data_final['month'].isin([12,1,2])).astype(int)
test_data_final['is_summer_month']=(test_data_final['month'].isin([6,7,8])).astype(int)

* chilled water readings are low during winter and high during summer.
* Hot water readings are low during summer and high during summer.

In [ ]:
test_data_final = reduce_mem_usage(test_data_final)

Mem. usage decreased to 2067.83 Mb (49.5% reduction)


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
            "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-25"]
h_date = pd.to_datetime(holidays,yearfirst=True)
test_data_final['is_pub_holiday'] = test_data_final['timestamp'].dt.date.isin(h_date.date).astype(int)
test_data_final['is_weekday'] = ((~test_data_final['timestamp'].dt.date.isin(h_date.date))&(test_data_final['weekday'].isin([0,1,2,3,4]))).astype(int)

* Energy requirements might be different from holidays to working days.

In [ ]:
#https://www.kaggle.com/c/ashrae-energy-prediction/discussion/124984

#horizsolar is supposed to calculate the solar horizontal radiation coming into the building.
latitude_dict = {0 :28.5383,1 :50.9097,2 :33.4255,3 :38.9072,4 :37.8715,5 :50.9097,6 :40.7128,7 :45.4215,8 :28.5383,9 :30.2672,10 :40.10677,11 :45.4215,12 :53.3498,13 :44.9375,14 :38.0293,15: 40.7128,}

test_data_final['latitude'] = test_data_final['site_id'].map(latitude_dict)
test_data_final['solarHour'] = (test_data_final['hour']-12)*15 
test_data_final['solarDec'] = -23.45*np.cos(np.deg2rad(360*(test_data_final['day']+10)/365)) 
test_data_final['horizsolar'] = np.cos(np.deg2rad(test_data_final['solarHour']))*np.cos(np.deg2rad(test_data_final['solarDec']))*np.cos(np.deg2rad(test_data_final['latitude'])) + np.sin(np.deg2rad(test_data_final['solarDec']))*np.sin(np.deg2rad(test_data_final['latitude']))
test_data_final['horizsolar'] = test_data_final['horizsolar'].apply(lambda x: 0 if x <0 else x)

test_data_final = test_data_final.drop(['latitude','solarHour','solarDec'], axis=1)
    # The sensible heat is a form of energy emitted into the atmosphere or absorbed
test_data_final['Sensible_Heat'] = 0.5274*(10.**(-4.))*test_data_final['square_feet']*(75.-test_data_final['air_temperature'])

test_data_final['relative_humidity'] = 100 * (np.exp((17.625 * test_data_final['dew_temperature']) / (243.04 + test_data_final['dew_temperature'])) / np.exp((17.625 * test_data_final['air_temperature'])/(243.04 + test_data_final['air_temperature'])))

* Humidity affects the energy requirements of the buildings. As humidity increases the load on the HVAC system increases due to which the meter readings are affected. so this feature will help to learn the energy requirements according to the humidity.


In [ ]:
#https://journals.sagepub.com/doi/pdf/10.1177/1550147719877616
#https://en.wikipedia.org/wiki/Wind_chill

test_data_final['discomfort_index'] = (1.8*test_data_final['air_temperature'] + 32) - ((0.55-0.0055*test_data_final['relative_humidity'])*(1.8 * test_data_final['air_temperature'] - 26))
test_data_final['wind_chill'] = 13.12 + 0.6215*test_data_final['air_temperature'] - 11.37*(test_data_final['wind_speed']**0.16) + 0.3965*test_data_final['air_temperature']*(test_data_final['wind_speed']**0.16)

* As the air_temperature falls, the chilling effect of any wind that is present increases.
* As index for measuring the discomfort felt in warm weather as a result of the combined effects of the temperature and humidity of the air.

In [ ]:
busy_hours = test_data_final.set_index(['timestamp']).between_time('06:00:00','18:00:00').reset_index()
busy_hours_timestamp = [i for i in busy_hours['timestamp']]
test_data_final['busy_hours'] = ((~test_data_final['timestamp'].dt.date.isin(h_date.date))&(test_data_final['timestamp'].isin(busy_hours_timestamp))).astype(int)

* As the meter_readings are high during day time than night.

In [ ]:
test_data_final = reduce_mem_usage(test_data_final)

Mem. usage decreased to 2783.62 Mb (9.1% reduction)


In [ ]:
test_data_final = test_data_final.reset_index()
test_data_final.to_feather('/content/drive/MyDrive/Ashrae_Great_Energy_Prediction/test_data_final.feather')

In [ ]:
test_data_final.head()

,index,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,month,air_temperature,hour,weekday,is_winter_month,is_summer_month,is_pub_holiday,is_weekday,horizsolar,Sensible_Heat,relative_humidity,days_in_month,hour_x,hour_y,day_x,day_y,month_x,month_y,discomfort_index,wind_chill,busy_hours
0,0,0,0,0,2017-01-01,0,0,7432,2008.0,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,1,15.601562,0,6,1,0,0,0,0.008896,23.281250,77.5625,31,0.0,1.0,-0.81543,0.579102,-0.987793,0.154297,59.8125,16.453125,0
1,1,1,1,0,2017-01-01,0,0,2720,2004.0,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,1,15.601562,0,6,1,0,0,0,0.008896,8.523438,77.5625,31,0.0,1.0,-0.81543,0.579102,-0.987793,0.154297,59.8125,16.453125,0
2,2,2,2,0,2017-01-01,0,0,5376,1991.0,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,1,15.601562,0,6,1,0,0,0,0.008896,16.843750,77.5625,31,0.0,1.0,-0.81543,0.579102,-0.987793,0.154297,59.8125,16.453125,0
3,3,3,3,0,2017-01-01,0,0,23685,2002.0,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,1,15.601562,0,6,1,0,0,0,0.008896,74.187500,77.5625,31,0.0,1.0,-0.81543,0.579102,-0.987793,0.154297,59.8125,16.453125,0
4,4,4,4,0,2017-01-01,0,0,116607,1975.0,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,1,15.601562,0,6,1,0,0,0,0.008896,365.250000,77.5625,31,0.0,1.0,-0.81543,0.579102,-0.987793,0.154297,59.8125,16.453125,0
